# K-Nearest Neighbors

Utilizar o modelo KNN para classificar frutas

Dataset público do Kaggle disponível em https://www.kaggle.com/mjamilmoughal/fruits-with-colors-dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dados = pd.read_csv('fruit_data_with_colors.txt')

In [ ]:
dados.head()

Pandas tem dificuldade em localizar o separador em arquivos TXT e portanto devemos fazer isso manualmente

In [ ]:
dados = pd.read_csv('fruit_data_with_colors.txt',sep='\t')

In [ ]:
dados.head()

In [ ]:
dados.info()

Vamos determinar o número de classes (tipos de frutas) e as respectivas quantidades

In [ ]:
sns.countplot(x='fruit_name',data=dados)

In [ ]:
dados['fruit_name'].value_counts()

Observamos que temos um dataset não balanceado devido a quantidade da fruta mandarim. Iremos descobrir se isto irá ou não afetar o nosso modelo

Verificando se existem NaNs ou elementos nulos na amostra

In [ ]:
dados.isna().sum()

In [ ]:
dados.isnull().sum()

Da análise observamos que tais elementos não existem

Obtendo os nomes das colunas

In [ ]:
colunas = dados.columns

In [ ]:
colunas

Removendo as colunas fruit_label e fruit_name

In [ ]:
colunas = colunas.drop(['fruit_label', 'fruit_name'])

In [ ]:
colunas

Normalizando os dados

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc=StandardScaler()

In [ ]:
dados[colunas.drop('fruit_subtype')] = sc.fit_transform(dados[colunas.drop('fruit_subtype')])

In [ ]:
dados.head()

A coluna fruit_subtype é uma variável categórica e precisa ser convertida para numérica

In [ ]:
from sklearn.preprocessing import LabelEncoder, label_binarize

In [ ]:
enconder = LabelEncoder()

In [ ]:
dados['fruit_subtype'] = enconder.fit_transform(dados['fruit_subtype'])

In [ ]:
classes = dados['fruit_label'].unique()
n_classes = len(classes)

In [ ]:
classes

In [ ]:
n_classes

In [ ]:
dados.head()

Separando nas variáveis X e Y

In [ ]:
X = dados.drop(['fruit_label','fruit_name'],axis=1).values
Y = dados['fruit_label'].values

In [ ]:
Y = label_binarize(Y, classes=classes)

Separando em amostras de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_treino,X_teste,Y_treino,Y_teste=train_test_split(X,Y,test_size=0.25,random_state=0)

Aplicando modelo KNN

No modelo KNN o número de vizinhos é uma variável de entrada. Para melhor determinar este número, iremos cacular a acurácia no intervalo entre 1 e 10.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
vizinhos = []
acc_treino = []
acc_teste = []
std = []

In [ ]:
for i in range(2,11):
    vizinhos.append(i)
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_treino,Y_treino)
    acc_treino.append(knn.score(X_treino,Y_treino))
    acc_teste.append(knn.score(X_teste,Y_teste))
    Y_previsto = knn.predict(X_teste)
    std.append(np.std(Y_previsto==Y_teste)/np.sqrt(Y_previsto.shape[0]))

Graficando acurácias em função do número de vizinhos

In [ ]:
plt.scatter(vizinhos,acc_treino,label='Amostra de treino',color='blue')
plt.scatter(vizinhos,acc_teste,label='Amostra de teste',color='red')
plt.xlabel('Número de vizinhos')
plt.ylabel('Acurácia')
plt.xlim([0,11])
plt.legend(loc='lower left')
plt.tight_layout()

Observamos que no intervalo 0<n_vizinhos<5 a acurácia de treino e teste é praticamente máxima. Por este motivo adotaremos n=4

In [ ]:
knn=KNeighborsClassifier(n_neighbors=4)

In [ ]:
knn.fit(X_treino,Y_treino)

In [ ]:
Y_previsto = knn.predict(X_teste)
y_prob = knn.predict_proba(X_teste)

Gerando matriz de confusão

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm=confusion_matrix(Y_teste.argmax(axis=1),Y_previsto.argmax(axis=1))

In [ ]:
cm

Calculando métricas

In [ ]:
from sklearn.metrics import classification_report,f1_score,precision_score,average_precision_score,recall_score,accuracy_score

Relatório de classificação

In [ ]:
cr = classification_report(Y_teste,Y_previsto,labels=[0,1])
print(cr)

F1-score

In [ ]:
f1_macro = f1_score(Y_teste,Y_previsto,average='macro')
print("F1 Macro score = {:0.2f}%".format(f1_macro*100))

f1_wei = f1_score(Y_teste,Y_previsto,average='weighted')
print("F1 Weighted score = {:0.2f}%".format(f1_wei*100))

Precision score

In [ ]:
precisao_macro = precision_score(Y_teste,Y_previsto,average='macro')
print("Precision score = {:0.2f}%".format(precisao_macro*100))

precisao_wei = precision_score(Y_teste,Y_previsto,average='weighted')
print("Precision score = {:0.2f}%".format(precisao_wei*100))

Recall score

In [ ]:
rec_macro = recall_score(Y_teste,Y_previsto,average='macro')
print("Recall score Macro = {:0.2f}%".format(rec_macro*100))

rec_wei = recall_score(Y_teste,Y_previsto,average='weighted')
print("Recall score Weighted = {:0.2f}%".format(rec_wei*100))

Accuracy score

In [ ]:
acc = accuracy_score(Y_teste,Y_previsto)
print("Accuracy score = {:0.2f}%".format(acc*100))

# Curva ROC

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score,auc
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_teste[:, i], Y_previsto[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])*100

In [ ]:
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],label=str(i)+' AUC='+str(roc_auc[i]))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

# Curva Precision-Recall

In [ ]:
from sklearn.metrics import precision_recall_curve,auc

In [ ]:
lr_precision = dict()
lr_recall = dict()
lr_auc = dict()
for i in range(n_classes):
    lr_precision[i], lr_recall[i], _ = precision_recall_curve(Y_teste[:, i], Y_previsto[:, i])
    lr_auc[i] = auc(lr_recall[i], lr_precision[i])*100

In [ ]:
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],label=str(i)+' AUC='+str(lr_auc[i]))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()